## Anomary Detection with timeseries_anomalies_stl 

data: splunk buttercup game added iplocation result

In [22]:
%pip install statsmodels --upgrade

     |████████████████████████████████| 9.7 MB 11.8 MB/s eta 0:00:01
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.1
    Uninstalling statsmodels-0.12.1:
      Successfully uninstalled statsmodels-0.12.1


In [51]:

!date
!python -V # check if python version is later than 3.8

import sys
print("sys.version\n",sys.version)

Mon May  8 13:18:46 JST 2023
Python 3.9.12
sys.version
 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]


In [52]:
import msticpy
print(msticpy.__version__)

2.4.0


In [ ]:
#mp.init_notebook()

In [139]:
import pandas as pd

df = pd.read_csv(
    "../data/buttercupgame_iplocation.csv", infer_datetime_format=True,
    usecols=["req_time","bytes","Country","status","action"]
    )
df["date_clock"] = pd.to_datetime(df["req_time"],  format="%d/%b/%Y:%H:%M:%S", errors='coerce').dt.strftime("%Y-%m-%d %H")
df['count'] = 1
df.head()

,Country,action,bytes,req_time,status,date_clock,count
0,Netherlands,addtocart,2744,08/Sep/2022:07:19:24,200,2022-09-08 07,1
1,Netherlands,purchase,1736,08/Sep/2022:07:19:26,200,2022-09-08 07,1
2,Netherlands,purchase,989,08/Sep/2022:07:19:25,200,2022-09-08 07,1
3,Netherlands,addtocart,1581,08/Sep/2022:07:19:24,200,2022-09-08 07,1
4,Netherlands,NaN,749,08/Sep/2022:07:19:24,200,2022-09-08 07,1


In [141]:


df_action = df.groupby(['date_clock','action'])['count'].sum().reset_index()
df_action.head()


,date_clock,action,count
0,2022-09-01 18,addtocart,17
1,2022-09-01 18,changequantity,7
2,2022-09-01 18,purchase,17
3,2022-09-01 18,remove,9
4,2022-09-01 18,view,20


In [142]:
df_action = df_action[df_action.action=="addtocart"]
df_action.head()

,date_clock,action,count
0,2022-09-01 18,addtocart,17
5,2022-09-01 19,addtocart,40
10,2022-09-01 20,addtocart,26
15,2022-09-01 21,addtocart,46
20,2022-09-01 22,addtocart,24


In [143]:
from msticpy.analysis.timeseries import timeseries_anomalies_stl
df_count = df_action[['date_clock','count']]
df_count = df_count.set_index('date_clock')
output = timeseries_anomalies_stl(df_count)
output.head()

,date_clock,count,residual,trend,seasonal,weights,baseline,score,anomalies
0,2022-09-01 18,17,-2,34,-14,1,19,-0.424389,0
1,2022-09-01 19,40,1,34,4,1,38,0.201091,0
2,2022-09-01 20,26,-3,34,-4,1,29,-0.632882,0
3,2022-09-01 21,46,3,34,8,1,42,0.618078,0
4,2022-09-01 22,24,-2,34,-7,1,26,-0.424389,0


In [144]:
output[output.anomalies == 1]

,date_clock,count,residual,trend,seasonal,weights,baseline,score,anomalies
94,2022-09-05 16,52,15,33,3,1,36,3.119998,1


In [146]:
from msticpy.nbtools.timeseries import display_timeseries_anomolies

# display_timeseries_anomolies関数の使用上、事前に結果のタイムフィールドにdatetime型を適用し、かつ時系列順に並び替える
output['date_clock'] = pd.to_datetime(output['date_clock'])
output = output.sort_values(by='date_clock')

timeseries_anomalies_plot = display_timeseries_anomolies(
    data=output, 
    y='count',
    time_column='date_clock'
    )

/var/folders/9j/5q4qwns11lv47mtj3ykbp5_40000gn/T/ipykernel_3437/3785166171.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) display_timeseries_anomolies. (Will be removed in version 2.0.0. Please use display_timeseries_anomalies) -- Deprecated since version 1.7.0.
  timeseries_anomalies_plot = display_timeseries_anomolies(


Loading BokehJS ...